This notebook is based on [***Getting started with text preprocessing***](https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing) by SRK on Kaggle.
More information on the [***dataset***](https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter).


## **Introduction**

In any machine learning task, cleaning or preprocessing the data is as important as model building if not more. And when it comes to unstructured data like text, this process is even more important. 

Objective of this kernel is to understand the various text preprocessing steps with code examples. 

Some of the common text preprocessing / cleaning steps are:
* Lower casing
* Removal of Punctuations
* Removal of Stopwords
* Removal of Frequent words
* Removal of Rare words
* Stemming
* Lemmatization
* Removal of emojis
* Removal of emoticons
* Conversion of emoticons to words
* Conversion of emojis to words
* Removal of URLs 
* Removal of HTML tags
* Chat words conversion
* Spelling correction


So these are the different types of text preprocessing steps which we can do on text data. But we need not do all of these all the times. We need to carefully choose the preprocessing steps based on our use case since that also play an important role. 

For example, in sentiment analysis use case, we need not remove the emojis or emoticons as it will convey some important information about the sentiment. Similarly we need to decide based on our use cases. 

In [59]:
import numpy as np
import pandas as pd 
import re # regular expressions => string parsing and filtering
import nltk # natural language toolkit => classification, tokenization, stemming, ...
import spacy # tokenizer, tagger, parser, NER, pretrained models
import string
pd.options.mode.chained_assignment = None


In [60]:
df = pd.read_csv("./data/tweets_preprocessing.csv")
df["text"] = df["text"].astype(str)
print(f"Shape: {df.shape}")
df.head()

Shape: (93, 7)


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


## **Lower Casing**

Lower casing is a common text preprocessing technique. The idea is to convert the input text into same casing format so that 'text', 'Text' and 'TEXT' are treated the same way. 

This is more helpful for text featurization techniques like frequency, tfidf as it helps to combine the same words together thereby reducing the duplication and get correct counts / tfidf values.

This may not be helpful when we do tasks like Part of Speech tagging (where proper casing gives some information about Nouns and so on) and Sentiment Analysis (where upper casing refers to anger and so on)

By default, lower casing is done my most of the modern day vecotirzers and tokenizers like [sklearn TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and [Keras Tokenizer](https://keras.io/preprocessing/text/). So we need to set them to false as needed depending on our use case. 

In [61]:
text_df = df[["text"]]

In [62]:
print("Before lowering:")
print(text_df.head().text.values)

Before lowering:
['@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡'
 '@105835 Your business means a lot to us. Please DM your name, zip code and additional details about your concern. ^RR https://t.co/znUu1VJn9r'
 "@76328 I really hope you all change but I'm sure you won't! Because you don't have to!"
 '@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 option 1, 4, 3 (Leave a message) to request a call back'
 "@VirginTrains see attached error message. I've tried leaving a voicemail several times in the past week https://t.co/NxVZjlYx1k"]


In [63]:
# Your code here:
# Convert the text_df text column to lowercase
text_df["text"] = text_df["text"].str.lower()

assert text_df.text.values[2] == "@76328 i really hope you all change but i'm sure you won't! because you don't have to!", "Your text is not lowered properly"

print("After lowering:")
print(text_df.head().text.values)
text_df.head()

After lowering:
['@applesupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡'
 '@105835 your business means a lot to us. please dm your name, zip code and additional details about your concern. ^rr https://t.co/znuu1vjn9r'
 "@76328 i really hope you all change but i'm sure you won't! because you don't have to!"
 '@105836 livechat is online at the moment - https://t.co/sy94vtu8kq or contact 03331 031 031 option 1, 4, 3 (leave a message) to request a call back'
 "@virgintrains see attached error message. i've tried leaving a voicemail several times in the past week https://t.co/nxvzjlyx1k"]


,text
0,@applesupport causing the reply to be disregar...
1,@105835 your business means a lot to us. pleas...
2,@76328 i really hope you all change but i'm su...
3,@105836 livechat is online at the moment - htt...
4,@virgintrains see attached error message. i've...


## **Removal of Punctuations**

One another common text preprocessing technique is to remove the punctuations from the text data. This is again a text standardization process that will help to treat 'hurray' and 'hurray!' in the same way.

We also need to carefully choose the list of punctuations to exclude depending on the use case. For example, the `string.punctuation` in python contains the following punctuation symbols 

`!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~`

We can add or remove more punctuations as per our need.

In [64]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [65]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text: str) -> str:
    """
    Removes punctuation characters from the input text

    Args:
        text (str): The input text from which punctuation characters will be removed

    Returns:
        str: A new string with all punctuation characters removed
    """
    # Your code here:
    # use the maketrans function to remove the punctuation specified in PUNCT_TO_REMOVE 
    # [https://www.w3schools.com/python/ref_string_maketrans.asp]
    translation_table = str.maketrans('','',PUNCT_TO_REMOVE)
    return text.translate(translation_table)

# now apply your function to the text column using the apply function [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html]
text_df["text_wo_punct"] = text_df["text"].apply(remove_punctuation)


In [66]:
assert text_df["text_wo_punct"].values[3] == "105836 livechat is online at the moment  httpstcosy94vtu8kq or contact 03331 031 031 option 1 4 3 leave a message to request a call back"
text_df.head()

,text,text_wo_punct
0,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...
1,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...
2,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...
3,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...
4,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...


## **Removal of stopwords**

Stopwords are commonly occuring words in a language like 'the', 'a' and so on. They can be removed from the text most of the times, as they don't provide valuable information for downstream analysis. In cases like Part of Speech tagging, we should not remove them as provide very valuable information about the POS.

These stopword lists are already compiled for different languages and we can safely use them. For example, the stopword list for english language from the nltk package can be seen below.


In [67]:
import nltk

In [68]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suying/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
print(nltk.data.path)

['/Users/suying/nltk_data', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/nltk_data', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/share/nltk_data', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/Users/suying/Downloads/WordNet-3.0 ', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/python3.11/site-packages/certifi/cacert.pem', '/Users/suying/Downloads/WordNet-3.0 ', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/python3.11/site-packages/certifi/cacert.pem']


In [70]:

STOPWORDS = ['the', 'to', 'be', 'in', 'and', 'under', 'is','a','i','you' ,'this','for','we','on','it','can','your','my','are','if','with']


Similarly we can also get the list for other languages as well and use them. 

In [71]:
sample = text_df.text.values[0]
sample

'@applesupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡'

In [72]:
split = sample.split(' ') # splits the input string into a list, the delimiter is a whitespace " "
split

['@applesupport',
 'causing',
 'the',
 'reply',
 'to',
 'be',
 'disregarded',
 'and',
 'the',
 'tapped',
 'notification',
 'under',
 'the',
 'keyboard',
 'is',
 'opened😡😡😡']

In [73]:
# Use list comprehension to remove the stopwords in split
# [https://www.programiz.com/python-programming/list-comprehension]
# Your list comprehension should be equivalent to:

# filtered_words = []  
# for word in split:
#   if word not in STOPWORDS:
#       filtered_words.append(word)


filtered_words = [word for word in split if word not in STOPWORDS]


assert filtered_words == ['@applesupport','causing','reply','disregarded','tapped','notification','keyboard','opened😡😡😡']

filtered_words

['@applesupport',
 'causing',
 'reply',
 'disregarded',
 'tapped',
 'notification',
 'keyboard',
 'opened😡😡😡']

In [74]:
filtered_string = " ".join(filtered_words) # the join function is useful to reconstruct filtered strings 
print(f"Before filtering: {sample}\nAfter filtering: {filtered_string}")

Before filtering: @applesupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡
After filtering: @applesupport causing reply disregarded tapped notification keyboard opened😡😡😡


In [75]:
def remove_stopwords(text: str) -> str:
    """
    Removes stopwords from the input text

    Args: 
        text (str): The input text from which stopwords will be removed

    Returns:
        str: A new string without stopwords
    """
    # Your code here:
    
    # Split the input text into a list of words
    words = text.split()

    # Remove stopwords from the list of words
    filtered_words = [word for word in words if word.lower() not in STOPWORDS]

    # Join the filtered words back into a single string
    result_text = " ".join(filtered_words)

    return result_text
   
   

text_df["text_wo_stop"] = text_df["text_wo_punct"].apply(remove_stopwords)
text_df.head()

,text,text_wo_punct,text_wo_stop
0,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...
1,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...
2,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope all change but im sure wont ...
3,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online at moment httpstcosy94v...
4,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...


## **Removal of Frequent words**

In the previous preprocessing step, we removed the stopwords based on language information. But say, if we have a domain specific corpus, we might also have some frequent words which are of lesser importance to us. 

So this step is to remove the frequent words in the given corpus. If we use something like tfidf (vectorizers are the topic of the second course), this is automatically taken care of.  

Let us get the most common words and then remove them in the next step

In [76]:
from collections import Counter
cnt = Counter()

# Your code here:
# Use the Counter class to return the most frequent words
# 1: join all the text in the text_wo_stop column using the join() function
all_text = " ".join(text_df["text_wo_stop"])
# 2: tokenize the text by using the split() function
tokenized_text = all_text.split()
# 3: instantiate the Counter class with your tokenized array
word_counter = Counter(tokenized_text)
# 4: use the most_common class method to return the most frequent words
most_common = word_counter.most_common()


#assert set(most_common[:10]) == set([('us', 25), ('dm', 19),('help', 17),('thanks', 12),('please', 10),('hi', 9),('ive', 8),('phone', 8),('version', 8),('get', 8)])
most_common[:10]

[('us', 25),
 ('dm', 19),
 ('help', 18),
 ('thanks', 13),
 ('of', 12),
 ('httpstcogdrqu22ypt', 12),
 ('so', 12),
 ('there', 12),
 ('applesupport', 11),
 ('please', 11)]

In [77]:


FREQWORDS = [w for (w, word_count) in most_common[:10]]

def remove_freqwords(text: str, freq_words: list=FREQWORDS) -> str:
    """
    Removes a selection of frequent words from the input string

    Inputs:
        text (str): The input text from which frequent words will be removed
        freq_words (list): A list of frequent words to remove from the text

    Returns:
        str: A new string with all frequent words removed
    """
    # Your code here:
    # Reuse your stopword function to filter out the 10 most frequent words
    # hint: you can write the whole function in one line using list comprehension equivalent to
    # words = []
    # for word, count in most_common[:10]:
    #   words.append(word)
    
    # Use a list comprehension to split the text into words and filter out frequent words
    filtered_words = [word for word in text.split() if word not in freq_words]

    # Join the filtered words back into a single string
    result_text = " ".join(filtered_words)

    return result_text
    


In [78]:
#assert remove_freqwords(text_df.text_wo_stop.values[1]) == "105835 business means lot name zip code additional details concern rr httpstcoznuu1vjn9r"

# Your code here:
# Apply your function to the text_wo_stop column
text_df["text_wo_stopfreq"] = text_df["text_wo_stop"].apply(remove_freqwords)

text_df.head()

,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...
1,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...,105835 business means lot name zip code additi...
2,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope all change but im sure wont ...,76328 really hope all change but im sure wont ...
3,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online at moment httpstcosy94v...,105836 livechat online at moment httpstcosy94v...
4,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...


## **Removal of Rare words**

This is very similar to previous preprocessing step but we will remove the rare words from the corpus. 

In [79]:
# let's keep only the latest version of the processed text
text_df = text_df[["text", "text_wo_stopfreq"]]
text_df.head()

,text,text_wo_stopfreq
0,@applesupport causing the reply to be disregar...,causing reply disregarded tapped notification ...
1,@105835 your business means a lot to us. pleas...,105835 business means lot name zip code additi...
2,@76328 i really hope you all change but i'm su...,76328 really hope all change but im sure wont ...
3,@105836 livechat is online at the moment - htt...,105836 livechat online at moment httpstcosy94v...
4,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...


In [80]:
# Your code here: 
# Reuse your previous most_common list to extract the 10 rarest words
RAREWORDS = [word for (word, word_count) in most_common[-10:]]

#assert set(RAREWORDS) == set(['lee', 'log', 'browser', 'including', 'httpstco9281okeebk105861', 'line', 'green', 'thin', 'keen', 'slowdown'])
RAREWORDS

['slowdown',
 'keen',
 'thin',
 'green',
 'line',
 'httpstco9281okeebk',
 'including',
 'browser',
 'log',
 'lee']

In [81]:
def remove_rarewords(text: str, rare_words: list=RAREWORDS) -> str:
    """
    Removes a selection of rare words from the input string

    Inputs:
        text (str): The input text from which rare words will be removed
        rare_words (list): A list of rare words to remove from the text

    Returns:
        str: A new string with all rare words removed
    """
    # Your code here:
    # Filter out the most frequent words from a text string
     # Use a list comprehension to split the text into words and filter out rare words
    filtered_words = [word for word in text.split() if word not in rare_words]

    # Join the filtered words back into a single string
    result_text = " ".join(filtered_words)

    return result_text
    

text_df["text_wo_stopfreqrare"] = text_df["text_wo_stopfreq"].apply(remove_rarewords)
text_df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,@applesupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...
1,@105835 your business means a lot to us. pleas...,105835 business means lot name zip code additi...,105835 business means lot name zip code additi...
2,@76328 i really hope you all change but i'm su...,76328 really hope all change but im sure wont ...,76328 really hope all change but im sure wont ...
3,@105836 livechat is online at the moment - htt...,105836 livechat online at moment httpstcosy94v...,105836 livechat online at moment httpstcosy94v...
4,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...


We can combine all the list of words (stopwords, frequent words and rare words) and create a single list to remove them at once.

In [82]:
import itertools
TO_REMOVE = [list(STOPWORDS), list(FREQWORDS), list(RAREWORDS)]

# Your code here:
# concatenate all the words to remove using itertools.chain [https://www.geeksforgeeks.org/python-itertools-chain/]
TO_REMOVE =[list(STOPWORDS), list(FREQWORDS), list(RAREWORDS)]

#assert len(TO_REMOVE) == (len(STOPWORDS) + len(FREQWORDS) + len(RAREWORDS))

len(TO_REMOVE)

3

In [83]:
def filter_text(text: str, words_to_remove :list=TO_REMOVE) -> str:
    """
    Removes a selection of words from the input string

    Inputs:
        text (str): The input text from which words will be removed
        words_to_remove (list): A list of words to remove from the text

    Returns:
        str: A new string with all listed words removed
        
    """
    words = text.split()

    # Filter out words that are in the list of words to remove
    filtered_words = [word for word in words if word not in words_to_remove]

    # Join the filtered words back into a single string
    result_text = " ".join(filtered_words)

    return result_text
     

text_df["filtered_text"] =  text_df.text.apply(filter_text)
text_df = text_df[['text', 'filtered_text']]
text_df.head()

,text,filtered_text
0,@applesupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...
1,@105835 your business means a lot to us. pleas...,@105835 your business means a lot to us. pleas...
2,@76328 i really hope you all change but i'm su...,@76328 i really hope you all change but i'm su...
3,@105836 livechat is online at the moment - htt...,@105836 livechat is online at the moment - htt...
4,@virgintrains see attached error message. i've...,@virgintrains see attached error message. i've...


## **Stemming**

Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form (From [Wikipedia](https://en.wikipedia.org/wiki/Stemming)).

This process is useful to **`reduce the vocabulary size`** by converting similar words to their root form.

For example, if there are two words in the corpus `walks` and `walking`, then stemming will stem the suffix to make them `walk`. But say in another example, we have two words `console` and `consoling`, the stemmer will remove the suffix and make them `consol` which is not a proper english word.

There are several type of stemming algorithms available and one of the famous one is porter stemmer (NLTK package) which is widely used.

In [84]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_demonstration(word: str) -> None:
    """
    Compares a word before and after stemming
    """
    print(f"Before stemming: {word}, after stemming: {stemmer.stem(word)}")

for word in ['programs', 'programming', 'programmed', 'walks', 'walked', 'walking', 'UPPERCASE', 'mistake', 'mistakke']:
    stem_demonstration(word)

Before stemming: programs, after stemming: program
Before stemming: programming, after stemming: program
Before stemming: programmed, after stemming: program
Before stemming: walks, after stemming: walk
Before stemming: walked, after stemming: walk
Before stemming: walking, after stemming: walk
Before stemming: UPPERCASE, after stemming: uppercas
Before stemming: mistake, after stemming: mistak
Before stemming: mistakke, after stemming: mistakk


In [85]:
def stem_words(text: str) -> str:
    """
    Applies the stemmer to an input string

    Args: 
        text (str): The text to be stemmed

    Returns:
        str: A new string where every word has been stemmed
    """
    # Your code here:
    # return a string where each word has been stemmed
     # Create a Porter Stemmer object
    stemmer = PorterStemmer()

    # Split the input text into words
    words = text.split()

    # Apply stemming to each word and store the stemmed words in a list
    stemmed_words = [stemmer.stem(word) for word in words]

    # Join the stemmed words back into a single string
    result_text = " ".join(stemmed_words)

    return result_text
    


In [86]:
#assert stem_words(text_df["filtered_text"].values[7]) == "@105836 work ok here, miriam. link https://t.co/0m2mph15eh ? ^mm" # note that the stemmer also applied lowercase

text_df["text_stemmed"] = text_df["filtered_text"].apply(stem_words)
text_df.head()

,text,filtered_text,text_stemmed
0,@applesupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,@applesupport caus the repli to be disregard a...
1,@105835 your business means a lot to us. pleas...,@105835 your business means a lot to us. pleas...,@105835 your busi mean a lot to us. pleas dm y...
2,@76328 i really hope you all change but i'm su...,@76328 i really hope you all change but i'm su...,@76328 i realli hope you all chang but i'm sur...
3,@105836 livechat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,@105836 livechat is onlin at the moment - http...
4,@virgintrains see attached error message. i've...,@virgintrains see attached error message. i've...,@virgintrain see attach error message. i'v tri...


In [87]:
all_text_no_stemming = ' '.join(text_df["text"]).split()
all_text_w_stemming = ' '.join(text_df["text_stemmed"]).split()

n_words_no_stemming = len(set(all_text_no_stemming))
n_words_w_stemming = len(set(all_text_w_stemming))
vocabulary_size_diff = n_words_no_stemming - n_words_w_stemming

#assert vocabulary_size_diff == 149

print(f"Number of unique words without stemming: {n_words_no_stemming}")
print(f"Number of unique words with stemming: {n_words_w_stemming}")
print(f"Difference: {vocabulary_size_diff} words")

Number of unique words without stemming: 813
Number of unique words with stemming: 766
Difference: 47 words


We can see that words like `private` and `propose` have their `e` at the end chopped off due to stemming. This is not intented. What can we do fort hat? We can use Lemmatization in such cases.

Also this porter stemmer is for English language. If we are working with other languages, we can use snowball stemmer. The supported languages for snowball stemmer are

In [88]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

## **Lemmatization**

Lemmatization is similar to stemming in reducing inflected words to their word stem but differs in the way that it makes sure the root word (also called as lemma) belongs to the language.
Here's a list of [examples](https://github.com/michmech/lemmatization-lists/blob/master/lemmatization-en.txt).

As a result, this one is generally slower than stemming process. So depending on the speed requirement, we can choose to use either stemming or lemmatization. 

Let us use the `WordNetLemmatizer` in nltk to lemmatize our sentences

In [89]:
import nltk
print(nltk.data.path)


['/Users/suying/nltk_data', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/nltk_data', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/share/nltk_data', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/Users/suying/Downloads/WordNet-3.0 ', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/python3.11/site-packages/certifi/cacert.pem', '/Users/suying/Downloads/WordNet-3.0 ', '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/python3.11/site-packages/certifi/cacert.pem']


In [90]:
import nltk

nltk.data.path.append("/Users/suying/Downloads/WordNet-3.0 ")


In [91]:
import certifi
import ssl
import nltk

ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(certifi.where())


In [92]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/suying/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [110]:
lemmatizer = WordNetLemmatizer()

def lem_demonstration(word: str, pos: str='n') -> None:
    print(f"Before lemmatization: {word}, after lemmatization: {lemmatizer.lemmatize(word, pos)}")

for word, pos in [('feet', 'n'), ('caring', 'v')]:
    lem_demonstration(word, pos)

Before lemmatization: feet, after lemmatization: foot
Before lemmatization: caring, after lemmatization: care


In [94]:

def lemmatize_words(text: str) -> str:
    """
    Applies lemmatization to the input string

    Args:   
        text (str): The input text to lemmatize

    Returns:
        str: The lemmatized version of the text
    """
    # Your code here:
    # return a string where each word has been lemmatized
    words = text.split()
    
    # 对每个单词进行词形还原
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # 将词形还原后的单词重新组合成文本
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

text_df["text_lemmatized"] = text_df["filtered_text"].apply(lemmatize_words)
text_df.head()

,text,filtered_text,text_stemmed,text_lemmatized
0,@applesupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,@applesupport caus the repli to be disregard a...,@applesupport causing the reply to be disregar...
1,@105835 your business means a lot to us. pleas...,@105835 your business means a lot to us. pleas...,@105835 your busi mean a lot to us. pleas dm y...,@105835 your business mean a lot to us. please...
2,@76328 i really hope you all change but i'm su...,@76328 i really hope you all change but i'm su...,@76328 i realli hope you all chang but i'm sur...,@76328 i really hope you all change but i'm su...
3,@105836 livechat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,@105836 livechat is onlin at the moment - http...,@105836 livechat is online at the moment - htt...
4,@virgintrains see attached error message. i've...,@virgintrains see attached error message. i've...,@virgintrain see attach error message. i'v tri...,@virgintrains see attached error message. i've...


We can see that the trailing `e` in the `propose` and `private` is retained when we use lemmatization unlike stemming. 

Wait. There is one more thing in lemmatization. Let us try to lemmatize `running` now.

In [95]:
lemmatizer.lemmatize("running")

'running'

Wow. It returned `running` as such without converting it to the root form `run`. This is because the lemmatization process depends on the POS tag to come up with the correct lemma. Now let us lemmatize again by providing the POS tag for the word. 

See this [table](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) for examples of POS tags.

In [96]:
lemmatizer.lemmatize("running", "v") # v for verb

'run'

Now we are getting the root form `run`. So we also need to provide the POS tag of the word along with the word for lemmatizer in nltk. Depending on the POS, the lemmatizer may return different results.

Let us take the example, `stripes` and check the lemma when it is both verb and noun.

In [97]:
print("Word is : stripes")
print("Lemma result for verb : ",lemmatizer.lemmatize("stripes", 'v'))
print("Lemma result for noun : ",lemmatizer.lemmatize("stripes", 'n'))

Word is : stripes
Lemma result for verb :  strip
Lemma result for noun :  stripe


Now let us redo the lemmatization process for our dataset.

In [98]:
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/suying/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [99]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text: str) -> str:
    """
    Apply lemmatization to the input string, considering words' POS tags.

    This function lemmatizes words in the input string based on their POS (Part-of-Speech) tags.
    
    Args:
        text (str): The input text to be lemmatized.

    Returns:
        str: A new string with lemmatized words.
    """
    # Initialize a mapping of POS tags to WordNet tags
    wordnet_map = {
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV,
        'J': wordnet.ADJ
    }

    # Your code here:
    # Use the nltk.pos_tag fucntion to get the POS tags of every word in the input
    # https://www.nltk.org/api/nltk.tag.pos_tag.html
    
    # You may also use nltk.word_tokenize to tokenize the text instead of split()
    # https://www.nltk.org/api/nltk.tokenize.html
    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)

    # Your code here:
    # Return the lemmatized version of the text
    # Inside the lemmatize function, use the (word, POS tag) tuple collected in the pos_tagged_text list
    # hint: query the wordnet_map (wordnet_map.get(... , ...)) using the pos tag, return wordnet.NOUN as a default
    lemmatized_words = []
    for word, pos_tag in pos_tags:
        # Get the first character of the POS tag (e.g., 'N' from 'NN' for noun)
        # Use the wordnet_map to convert it to WordNet tag (default to noun if not found)
        wordnet_tag = wordnet_map.get(pos_tag[0], wordnet.NOUN)
        # Perform lemmatization
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_tag)
        lemmatized_words.append(lemmatized_word)

    # Reconstruct the lemmatized text
    lemmatized_text = ' '.join(lemmatized_words)

    return lemmatized_text


In [109]:
import nltk
nltk.download('wordnet')
nltk.data.path.append("/Users/suying/nltk_data ")


[nltk_data] Downloading package wordnet to /Users/suying/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [111]:
#assert lemmatize_words("Any man who must say 'I am the king' is no true king.") == "Any man who must say ' I be the king ' be no true king ."
text_df["text_lemmatized"] = text_df["filtered_text"].apply(lambda text: lemmatize_words(text))

text_df.head()

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/suying/nltk_data'
    - '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/nltk_data'
    - '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/share/nltk_data'
    - '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/suying/Downloads/WordNet-3.0 '
    - '/Users/suying/Library/Caches/pypoetry/virtualenvs/nlp-courses-vId43bEJ-py3.11/lib/python3.11/site-packages/certifi

In [112]:
all_text_no_lemm = ' '.join(text_df["text"]).split()
all_text_w_lemm = ' '.join(text_df["text_lemmatized"]).split()

n_words_no_lemm = len(set(all_text_no_lemm))
n_words_w_lemm = len(set(all_text_w_lemm))
vocabulary_size_diff = n_words_no_lemm - n_words_w_lemm

print(f"Number of unique words without stemming: {n_words_no_lemm}")
print(f"Number of unique words with stemming: {n_words_w_lemm}")
print(f"Difference: {vocabulary_size_diff} words out of {df.shape[0]} sample")

#assert vocabulary_size_diff == 216

Number of unique words without stemming: 813
Number of unique words with stemming: 795
Difference: 18 words out of 93 sample


We can now see that in the third row, `sent` got converted to `send` since we provided the POS tag for lemmatization.

## **Removal of Emojis**

With more and more usage of social media platforms, there is an explosion in the usage of emojis in our day to day life as well. Probably we might need to remove these emojis for some of our textual analysis.

Thanks to [this code,](https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b) please find below a helper function to remove emojis from our text. 

In [121]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text: str) -> str:
    """
    Removes emojis from the input text

    Args: 
        text (str): The input text to remove emojis from

    Returns:
        str: A next text without emojis
    """
    # define a regular expression pattern
    emoji_pattern = re.compile("[" 
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"  # Miscellaneous symbols
                           u"\U000024C2-\U0001F251"  # Enclosed characters
                           "]+", flags=re.UNICODE)   # '+' signifies that those characters can occur once or more consecutively

    # Your code here:
    # Use the sub() function to remove emoji_pattern from the text
    cleaned_text = emoji_pattern.sub(r'', text)
    # https://www.pythontutorial.net/python-regex/python-regex-sub/
    return cleaned_text


In [119]:
import unicodedata


In [122]:
assert remove_emoji("game is on 🔥🔥") == 'game is on '


In [123]:
assert remove_emoji("game is on 🔥🔥") == 'game is on '
remove_emoji("Regular expressions are so much fun.😂") # a laughing emoji clearly doesn't belong in this sentence

'Regular expressions are so much fun.'

`/!\ Be aware of the patterns you use with regular expresions /!\`

Example: The United States flag emoji "🇺🇸" is part of the Unicode range \U0001F1E0-\U0001F1FF, which is included in the pattern. 

Therefore, when you use this pattern to remove emoji characters, it will also remove the flag emoji "🇺🇸."

`This might or might not be a problem depending on your use case, but you have to be aware of the design decisions you are making.`

In [125]:
remove_emoji("This is a 😀 sample text with 🚀 emojis 🇺🇸")

'This is a  sample text with  emojis '

## **Removal of Emoticons**

This is what we did in the last step right? >Not exactly. We did remove emojis in the last step but not emoticons. There is a minor difference between emojis and emoticons. 

From Grammarist.com, emoticon is built from keyboard characters that when put together in a certain way represent a facial expression, an emoji is an actual image.

:-) is an emoticon

😀 is an emoji

Thanks to [NeelShah](https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py) for the collection of emoticons, we are going to use them to remove emoticons. 

Please note again that the removal of emojis / emoticons are not always preferred and decision should be made based on the use case at hand.

In [132]:
from utils import emoticons
EMOTICONS = emoticons()
print(list(EMOTICONS.items())[:5])

[(':‑\\)', 'Happy face or smiley'), (':\\)', 'Happy face or smiley'), (':-\\]', 'Happy face or smiley'), (':\\]', 'Happy face or smiley'), (':-3', 'Happy face smiley')]


In [133]:
def remove_emoticons(text):
    # You don't have to learn this regular expression, this is just an example of 
    # how tedious writting regex can be
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Hello :-)")

'Hello '

In [134]:
remove_emoticons("I am sad :(")

'I am sad '

## **Conversion of Emoticon to Words**

In the previous step, we have removed the emoticons. In case of use cases like sentiment analysis, the emoticons give some valuable information and so removing them might not be a good solution. What can we do in such cases?

One way is to convert the emoticons to word format so that they can be used in downstream modeling processes. Thanks for Neel again for the wonderful dictionary that we have used in the previous step. We are going to use that again for conversion of emoticons to words. 


`Regex breakdown:`
```Python
u'('+emot+')'
```
* `u` in front of a string indicates that the string contains Unicode characters
* `'('` and `')'`: These are regular characters, not special in any way. They are just literal parentheses
*  `emot`: The string variable representing the emoji, for instance `:‑\)`
* `+`: This is the string concatenation operator. It combines the characters and the value of the `emot` variable together to create a new string.

So, when you see `u'('+emot+')'`, it's creating a Unicode string that contains a left parenthesis `'('`, the value of the `emot` variable (which is a placeholder for the text or pattern you want to find), and a right parenthesis `')'`. 

We'll use this pattern in the next cell to replace the emojis within strings: for example "Hi :-)" => "Hi Happy_face_smiley"

In [142]:
def convert_emoticons(text :str) -> str:
    """
    Convert emoticons in the input text to their corresponding textual descriptions.

    This function takes the input text and searches for known emoticons within it. 
    For each detected emoticon, it replaces it with its cleaned textual description.

    Args:
        text (str): The input text containing emoticons to be converted.

    Returns:
        str: A new string with emoticons replaced by their textual descriptions.

    Example:
        >>> convert_emoticons("I'm feeling happy 😊 and excited! 😀")
        "I'm feeling happy Smiling_Face_with_Smiling_Eyes and excited! Grinning_Face_with_Big_Eyes"
    """
    EMOTICONS = {
        ":-)": "Smiling",
        ":)": "Smiling",
        ":-(": "Frown_sad_andry_or_pouting",
        ":(": "Frown_sad_andry_or_pouting",
        # Add more emoticons and descriptions as needed
    }
   

    # Define a regular expression pattern to match emoticons
    emoticon_pattern = re.compile(r'(' + '|'.join(re.escape(key) for key in EMOTICONS.keys()) + r')')

    # Replace emoticons in the text with their descriptions
    def replace_emoticon(match):
        return EMOTICONS[match.group(0)]

    result_text = emoticon_pattern.sub(replace_emoticon, text)

    return result_text
    # Your code here:
    # iterate over the EMOTICONS dictionary items
    #for emoticon, description in EMOTICONS.items():
        #text = text.replace(emoticon, description)

    # for each description, replace comas "," by empty strings ""
    
    #text = text.replace(",", "")
    # split the string into a list using split()
    # join the remaining words with an underscore "_"
    # Ex: 'Laughing, big grin or laugh with glasses' => 'Laughing_big_grin_or_laugh_with_glasses'
# 将文本分割成单词列表
    #words = text.split()

    # 使用下划线连接剩余的单词
    #joined_description = "_".join(words)

    # 使用正则表达式替换表情符号
    #pattern = r'(' + '|'.join(re.escape(emoticon) for emoticon in EMOTICONS.keys()) + r')'
    
    # finally, use re.sub() to replace the emoji_pattern by your joined description within the text
    #result_text = re.sub(pattern, joined_description, text)

    # hint: use the pattern described above
    
    
    

    #return result_text

#assert convert_emoticons("I am sad :(") == 'I am sad Frown_sad_andry_or_pouting'

text = "Hello :-) :-)"
convert_emoticons(text)

'Hello Smiling Smiling'

This method might be better for some use cases when we do not want to miss out on the emoticon information.

## **Conversion of Emoji to Words**

Now let us do the same for Emojis as well. Neel Shah has put together a list of emojis with the corresponding words as well as part of his [Github repo](https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py). We are going to make use of this dictionary to convert the emojis to corresponding words.

Again this conversion might be better than emoji removal for certain use cases. Please use the one that is suitable for the use case. 

In [1]:
from utils import emojis_unicode

EMO_UNICODE = emojis_unicode()
print(f"Emoticon unicode: {list(EMO_UNICODE.items())[:5]}")

# reversing the dictionary for facilitated quer
UNICODE_EMO = {v: k for k, v in EMO_UNICODE.items()}

Emoticon unicode: [(':1st_place_medal:', '🥇'), (':2nd_place_medal:', '🥈'), (':3rd_place_medal:', '🥉'), (':AB_button_(blood_type):', '🆎'), (':ATM_sign:', '🏧')]


In [144]:
print(UNICODE_EMO['😂'])
print(UNICODE_EMO['🥇'])

:face_with_tears_of_joy:
:1st_place_medal:


Let's clean the emoji descriptions before using them:

In [5]:
def convert_emojis(text :str) -> str:
    """
    Convert emojis in the input text to their corresponding textual descriptions.

    This function takes the input text and searches for known emojis within it. 
    For each detected emoji, it replaces it with its cleaned textual description.

    Args:
        text (str): The input text containing emojis to be converted.

    Returns:
        str: A new string with emojis replaced by their textual descriptions.

    Example:
        >>> convert_emojis("I'm feeling 😊 and excited! 😀")
        "I'm feeling Smiling_Face_with_Smiling_Eyes and excited! Grinning_Face_with_Big_Eyes"
    """
    # Your code here:
    # This is almost identical to the convert_emoticons() function
    # iterate over the EMO_UNICODE dictionary items
    # for each description, replace comas "," by empty strings ""
    # split the string into a list using split()
    # join the remaining words with an underscore "_"
    # Ex: 'Laughing, big grin or laugh with glasses' => 'Laughing_big_grin_or_laugh_with_glasses'
    # finally, use text.replace to replace the emoji_pattern by your joined description within the text
    
    EMO_UNICODE = {
        '😀': 'Grinning_Face',
        '😃': 'Grinning_Face_with_Big_Eyes',
        '😄': 'Grinning_Face_with_Smiling_Eyes',
        '😁': 'Beaming_Face_with_Smiling_Eyes',
        '😆': 'Grinning_Squinting_Face',
        '🔥': 'fire',
        '😂': 'face_with_tears_of_joy'
        # Add more emoji descriptions as needed
    }

    # Iterate over the EMO_UNICODE dictionary items
    for emoji, description in EMO_UNICODE.items():
        # Replace commas "," in descriptions with empty strings ""
        description = description.replace(",", "")
        # Split the description into words using split()
        words = description.split()
        # Join the remaining words with an underscore "_"
        joined_description = "_".join(words)
        # Use text.replace to replace the emoji with the joined description within the text
        text = text.replace(emoji, joined_description)

    return text

     

text = "game is on 🔥"

print(convert_emojis(text))
assert convert_emojis(text) == 'game is on fire'

game is on fire


In [7]:
text = "Hilarious 😂"
print(convert_emojis(text))
assert convert_emojis(text) == 'Hilarious face_with_tears_of_joy'

Hilarious face_with_tears_of_joy


## **Removal of URLs**

Next preprocessing step is to remove any URLs present in the data. For example, if we are doing a twitter analysis, then there is a good chance that the tweet will have some URL in it. Probably we might need to remove them for our further analysis. 

We can use the below code snippet to do that.

`Regex breakdown:`
```Python
r'https?://\S+|www\.\S+'
# could also be understood as 
(r'https?://\S+') or (r'www\.\S+')
```
* `r` in front of a string indicates that Python shall treat the string as a raw litteral (avoids `\` being treated as escape characters)
* `https?://'`: This part of the regular expression matches URLs that start with either "http://" or "https:////". The `s?` portion allows for an optional "s" character, so it matches both "http://" and "https://".
*  `\S+`: This part of the regular expression matches one or more non-whitespace characters. It's used to match the domain part of the URL (e.g., www.example.com).
|: This is the alternation operator, which acts like an OR operator in regular expressions. It allows you to match either the pattern on the left or the pattern on the right. In this case, it's used to match either URLs starting with "http://" or "https://", or URLs starting with "www.".
* `www\.\S+`: This part of the regular expression matches URLs that start with "www." and then followed by one or more non-whitespace characters. It's commonly used to match URLs like "www.example.com".

In summary, this regular expression is designed to identify and capture URLs in a text string, whether they start with `"http://"`, `"https://"`, or `"www."`. It's a common pattern for extracting or hyperlinking URLs in text processing tasks.
So, when you see `u'('+emot+')'`, it's creating a Unicode string that contains a left parenthesis `'('`, the value of the `emot` variable (which is a placeholder for the text or pattern you want to find), and a right parenthesis `')'`. 

In [147]:
def remove_urls(text :str) -> str:
    """
    Remove URLs (web links) from the input text.

    This function searches for URLs in the input text and removes them, leaving the
    text without any web links.

    Args:
        text (str): The input text from which URLs will be removed.

    Returns:
        str: A new string with URLs removed.

    Example:
        >>> remove_urls("Visit our website at https://www.example.com to learn more.")
        "Visit our website at to learn more."
    """
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

Let us take a `https` link and check the code

In [148]:
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

'Driverless AI NLP blog post on '

Now let us take a `http` url and check the code

In [149]:
text = "Please refer to link http://lnkd.in/ecnt5yC for the paper"
remove_urls(text)

'Please refer to link  for the paper'

Thanks to Pranjal for the edge cases in the comments below. Suppose say there is no `http` or `https` in the url link. The function can now captures that as well.

In [150]:
text = "Want to know more. Checkout www.h2o.ai for additional information"
remove_urls(text)

'Want to know more. Checkout  for additional information'

## **Removal of HTML Tags**

One another common preprocessing technique that will come handy in multiple places is removal of html tags. This is especially useful, if we scrap the data from different websites. We might end up having html strings as part of our text. 

First, let us try to remove the HTML tags using regular expressions. 

`Regex breakdown:`
```Python
'<.*?>'
```
* `<` and `>`: simply match the opening and closing brackets of HTML tags, e.g. \<div>
* `.*?`: This is the `non-greedy` or `lazy quantifier` *?, which matches any character (represented by `.` ) zero or more times, but it does so as few times as possible to make a valid match. In the context of HTML tags, this means it will match the shortest possible sequence of characters between the opening < and closing > tags.

So, the entire regular expression `'<.*?>'` is used to match and capture the shortest possible HTML tag found in a text string. This is useful in cases where you want to extract or remove HTML tags from a text while preserving the shortest possible tag structure.

In [151]:
def remove_html(text :str) -> str:
    """
    Remove HTML tags and content from the input text.

    This function searches for HTML tags within the input text and removes them, 
    leaving only the plain text content.

    Args:
        text (str): The input text containing HTML tags to be removed.

    Returns:
        str: A new string with HTML tags and content removed.

    Example:
        >>> remove_html("<p>This is <b>bold</b> text.</p>")
        "This is bold text."
    """
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

text = """<div>
<h1> H2O</h1>
<SomeComponent/>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html('text'))

text


We can also use `BeautifulSoup` package to get the text from HTML document in a more elegant way.

In [152]:
from bs4 import BeautifulSoup

def remove_html(text :str) -> str:
    """
    Remove HTML tags and content from the input text using BeautifulSoup.

    This function utilizes the BeautifulSoup library to parse the input text as HTML
    and then extracts and returns the plain text content, removing all HTML tags.

    Args:
        text (str): The input text containing HTML tags to be removed.

    Returns:
        str: A new string with HTML tags and content removed.

    Example:
        >>> remove_html("<p>This is <b>bold</b> text.</p>")
        "This is bold text."
    """
    return BeautifulSoup(text, "lxml").text

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>
"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI




## **Chat Words Conversion**

This is an important text preprocessing step if we are dealing with chat data. People do use a lot of abbreviated words in chat and so it might be helpful to expand those words for our analysis purposes. 

Got a good list of chat slang words from this [repo](https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt). We can use this for our conversion here. We can add more words to this list.

In [153]:
from utils import slang_words

slang_words_list = slang_words()
print(list(slang_words_list.items())[:10])

[('AFAIK', 'As Far As I Know'), ('AFK', 'Away From Keyboard'), ('ASAP', 'As Soon As Possible'), ('ATK', 'At The Keyboard'), ('ATM', 'At The Moment'), ('A3', 'Anytime, Anywhere, Anyplace'), ('BAK', 'Back At Keyboard'), ('BBL', 'Be Back Later'), ('BBS', 'Be Back Soon'), ('BFN', 'Bye For Now')]


In [ ]:
chat_words_list = list(slang_words_list.keys())

def chat_words_conversion(text: str) -> str:
    slang_words_list = slang_words()
    chat_words_list = list(slang_words_list.keys())
    new_text = []

    # Your code here: 
    # iterate through the text
        # if the uppercase version of a word is contained in chat_word_list
        # get the translation from the slang_word_list and add it to new_text
        # otherwise add the word to new_text
    # return the joined text

    return 

assert chat_words_conversion("one minute BRB") == 'one minute Be Right Back'


In [ ]:
chat_words_conversion("imo this is awesome")

We can add more words to our abbreviation list and use them based on our use case. 

## **Spelling Correction**

One another important text preprocessing step is spelling correction. Typos are common in text data and we might want to correct those spelling mistakes before we do our analysis. 

If you are interested in writing a spell corrector of our own, take a look at [How to Write a Spelling Corrector](https://norvig.com/spell-correct.html) from Peter Norvig.

For the sake of brevity, let's use the python package `pyspellchecker` for our spelling correction.

In [154]:
# %pip install pyspellchecker
from spellchecker import SpellChecker

In [155]:
def correct_spellings(text: str) -> str:
    """
    Correct spelling errors in the input text using a spell checker.

    This function identifies and corrects spelling errors in the input text by utilizing a spell checker
    (presumably the `spell` object). It splits the input text into words, identifies misspelled words,
    and replaces them with their corrected versions. The corrected text is then returned.

    Args:
        text (str): The input text with possible spelling errors.

    Returns:
        str: A new string with spelling errors corrected.

    Example:
        >>> correct_spellings("Ths is an exmple of misspeled wrds.")
        "This is an example of misspelled words."
    """
    spell = SpellChecker()
    corrected_text = []
    # Your code here:
    # use the spell.unknown() function to identify misspelled words in the text
    #   https://pyspellchecker.readthedocs.io/en/latest/quickstart.html
    # iterate through the text
    # if the current word is misspelled 
    #   append the corrected version to corrected_text
    # otherwise append the original word
    # return the joined version of the text
    # Split the input text into words
    words = text.split()

    # Iterate through each word in the text
    for word in words:
        # Check if the current word is misspelled
        if word in spell.unknown(words):
            # If misspelled, correct it and append to the corrected_text list
            corrected_word = spell.correction(word)
            corrected_text.append(corrected_word)
        else:
            # If not misspelled, append the original word
            corrected_text.append(word)

    # Join the corrected words to form the corrected text
    corrected_text = ' '.join(corrected_text)

    return corrected_text

     
        

assert correct_spellings("speling correcton") == 'spelling correction'

text = "Hopefully you larned smething durng th classn, seeee you in twwo wekks !"
correct_spellings(text)

'Hopefully you learned something during the class see you in two weeks !'

## ***Your thoughts here:***

TODO:
* Summarize what you have learned and remembered from the class
* Why is a preprocessing pipeline important ? 
* What are the differences about raw and preprocessed text from the persepctive of an ML model ?
* Anything else you want to comment on (see paragraph below for ideas)
* Enjoy the rest of your day

Summarize what you have learned and remembered from the class

NLP It involves cleaning and transforming text data to make it suitable for analysis and modeling.
Common Preprocessing Steps: Some common preprocessing steps include lowercasing, tokenization, removing punctuation, removing stop words, stemming or lemmatization, and handling special characters or symbols. <br>
Tokenization: Tokenization is the process of splitting text into individual words or tokens. It is a fundamental step in text preprocessing and helps in creating a structured representation of text data.<br>
Stop Words: Stop words are common words (e.g., "the," "and," "in") that are often removed from text data because they don't carry significant meaning for many NLP tasks.
Stemming vs. Lemmatization: Stemming and lemmatization are techniques used to reduce words to their base or root forms. <br>
Handling Emoticons and Emojis: Emoticons and emojis can convey emotions in text, and preprocessing can involve replacing them with their textual descriptions to make them more interpretable.<br>

Why is a preprocessing pipeline important ?

It is important because it helps prepare raw text data for machine learning models. It ensures that the data is in a clean and structured format, making it easier for models to extract meaningful patterns and features.


What are the differences about raw and preprocessed text from the persepctive of an ML model ?

Raw text often contains a lot of noise, including punctuation, stop words, and special characters. Preprocessed text eliminates this noise, making it easier for models to focus on relevant information.<br>
Preprocessed text may involve techniques like tokenization, which converts text into a structured format that models can work with.

## **End of the mandatory section**

***Valued in the notation:***
* Your code should be shareable with your colleagues (clean, commented, reusable, functional)
* This pipeline is not perfect, text preprocessing is a difficult task requiring design decisions. You should be aware and comment on the different limits of your code and pipeline. For example:
  * Does the order of preprocessing steps matter ?
  * What design choices were made in this notebook, what risks do we accept by using it ?
  * Are there use cases (for example tasks or types of datasets) that are more or less adapted to the way we approach preprocessing ? Should we then remove or add some steps ?
  * Anything you want to comment on ...  


***To get the `advanced` grade: Complete the `bonus_exercise_cleaning.py` script and combine the different functions we've seen in this notebook in a nicely written pipeline and clean the text examples provided in `to_clean.csv`. Feel free to add any new step of your choice and to chain the different processing steps in any order that makes sense to you (of course, `comment on those decisions`).***

*You can use an Sklearn pipeline ([docs](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)) along with FunctionTransformer ([docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html)) to nicely chain the functions we wrote, we will reuse this tool in the next TP to add feature extraction functions and provide the data to machine learning models.*

<br>

### ***If you have any additional questions or feedback on the course and practical works, don't hesitate emailing me at ryan.pegoud@epfedu.fr***